In [34]:
import json
import numpy as np
import pandas as pd
import glob
import re
from tqdm import tqdm
import os.path
import collections
from collections import defaultdict


In [2]:
def remove_newlines(d: dict) -> dict:
    """
    Removes newline characters from string values in a dictionary, and converts any 'None of the above' values to a list
    with a single element 'None of the above'. Also strips square brackets from values that are lists and splits them
    into separate string elements.
    Args:
        d: The input dictionary to process.

    Returns:
        The processed dictionary with newlines removed and 'None of the above' values as lists.

    """
    for k, v in d.items():
        d[k] = v.replace('\n', '').strip()
        if d[k] == 'None of the above.':
            d[k] = ['None of the above']
        else:
            d[k] = [val.strip() for val in d[k].strip('[]').split(',')]
    return d


def strip_quotes(d: dict) -> dict:
    """
    Removes quotes around string values in a dictionary, and converts lists of quoted string values into lists of
    unquoted string values.
    Args:
        d: The input dictionary to process.

    Returns:
        The processed dictionary with quotes removed from string values and lists of unquoted string values.
    """
    new_dict = {}
    for k, v in d.items():
        if isinstance(v, list):
            new_list = []
            for item in v:
                if isinstance(item, str):
                    new_list.append(item.strip().strip("'"))
                else:
                    new_list.append(item)
            new_dict[k] = new_list
        else:
            new_dict[k] = v.strip().strip("'")
    return new_dict

def extract_name_and_file(path):
    filename = os.path.basename(path)
    name, ext = os.path.splitext(filename)
    species_name = path.split('/')[-2]
    return species_name, name

def split_on_capitals(string):
    # Split the string at every capital letter
    split_string = re.findall('[A-Z][^A-Z]*', string)
    # Join the split string with spaces
    new_string = ' '.join(split_string)
    
    return new_string



### Open Data

In [6]:
folder_traits = "../../../data/OpenAI/Traits/"

with open(F"{folder_traits}Andrei.json", 'r') as f:
  caribbean_traits_dict = json.load(f)

In [60]:
# Define the path where the prompt results are stored
folder_prompts = "../../../data/OpenAI/PromptsResults/"

# Get a list of all folders in the path
species_folders = glob.glob(F"{folder_prompts}*")

# Create a defaultdict to store the prompt results for each species
prompt_results_dict = collections.defaultdict(dict)

# Loop through each species folder
for idx, species_folder in enumerate(species_folders):
    # Extract the species name from the folder path
    species_name = species_folder[36:60].replace('_', ' ')
    
    # Get a list of all JSON files in the folder
    json_list = glob.glob(F"{species_folder}/*")

    # Loop through each JSON file
    for json_file in (pbar := tqdm(json_list, leave=False, position=0)):
        # Update the progress bar description
        pbar.set_description(f"{idx}: {species_name}")

        # Create an empty dictionary to store the trait results
        trait_dict = {}

        # Extract the trait name and file name from the JSON file path
        _, trait = extract_name_and_file(json_file)
        # Clean up the trait name by splitting on capital letters
        trait = split_on_capitals(trait)

        # Read in the JSON file
        with open(json_file, 'r') as f:
            prompt_result = json.load(f)

        # Extract the prompt result for the trait and add it to the trait dictionary
        trait_dict[trait] = prompt_result['choices'][0]['message']['content']
        # Clean up the trait dictionary by removing newlines and quotes
        trait_dict = remove_newlines(trait_dict)
        trait_dict = strip_quotes(trait_dict)

        # print('asda, ' , trait_dict)

        # Add the trait dictionary to the list of prompt results for the species
        trait_name = list(trait_dict.keys())[0]
        trait_name_cap = trait_name.capitalize()
        # print(trait_dict[trait_name])
        prompt_results_dict[species_name][trait_name_cap] = trait_dict[trait_name] # Contains a list


In [61]:
prompt_results_dict

defaultdict(dict,
            {'Amyris ignea': {'Petals corolla shape': ['tubular'],
              'Petals corolla colour': ['yellow',
               'white',
               'pink',
               'white',
               'pink',
               'purple'],
              'Sepals calyx shape': ['None of the above'],
              'Leaf position': ['None of the above'],
              'Leaf margin': ['crenate', 'serrate'],
              'Inflorescence type': ['panicle'],
              'Seed colour': ['None of the above'],
              'Leaf glands': ['translucent oil cells'],
              'Stipules': ['None of the above (Information about the Stipules trait was not mentioned in the text.)'],
              'Sepals calyx numer': ['4-5'],
              'Fruit colour': ['green'],
              'Leaf upper side': ['glabrous', 'pubescent'],
              'Thornsspines': ['absent'],
              'Life form': ['tree'],
              'Leaf rachis': ['None of the above'],
              'Leaf shape'

### Dataframe with GT

In [8]:
root = "../../../data/OpenAI/DataFrames/"

file = "DF_Andrei.csv"
df_Andrei = pd.read_csv(F"{root}{file}", header=[0, 1], index_col=0)\
    .rename_axis('Species', axis='index')
df_Andrei_species = list(df_Andrei.index)


file = "DF_Daniel.csv"
df_Daniel = pd.read_csv(F"{root}{file}", header=[0, 1], index_col=0)\
    .rename_axis('Species', axis='index')\
    .dropna()
df_Daniel_species = list(df_Daniel.index)

file = "DF_Pierre.csv"
df_Pierre = pd.read_csv(F"{root}{file}", header=[0, 1], index_col=0)\
    .rename_axis('Species', axis='index')
df_Pierre_species = list(df_Pierre.index)

In [11]:
df_Andrei[["Life form", "Leaf position"]].head(2)

Life form      Leaf position                               \
                        liana tree     alternate alternate, opposite opposite   
Species                                                                         
Avicennia germinans         0    1             0                   0        1   
Metopium brownei            0    1             1                   0        0   

                                                                            
                    opposite, whorls of 3 opposite, whorls of 3, alternate  
Species                                                                     
Avicennia germinans                     0                                0  
Metopium brownei                        0                                0

In [92]:
df = pd.DataFrame.from_dict(prompt_results_dict, orient='index')
# df = df.apply(pd.Series.explode)
df[["Petals corolla shape", "Petals corolla colour"]].head(5)

,Petals corolla shape,Petals corolla colour
Amyris ignea,[tubular],"[yellow, white, pink, white, pink, purple]"
Avicennia germinans,"[rounded, tubular]",[white]
Bursera simaruba,[None of the above],[None of the above]
Bourreria succulenta,[salverform],[white]


In [94]:
for col in df.columns:
    df = df.explode(col)

In [106]:
df.drop_duplicates()

,Petals corolla shape,Petals corolla colour,Sepals calyx shape,Leaf position,Leaf margin,Inflorescence type,Seed colour,Leaf glands,Stipules,Sepals calyx numer,...,Leaf rachis,Leaf shape,Fruit shape,Stamen shape,Petals corolla number,Stamen number,Leaf lower side,Fruit type,Aril colour,Leaf composition
Amyris ignea,tubular,yellow,None of the above,None of the above,crenate,panicle,None of the above,translucent oil cells,None of the above (Information about the Stipu...,4-5,...,None of the above,elliptic,ovoid,None of the above,5,10,None of the above (the 'Leaf lower side' trait...,capsule,None of the above. The text did not mention an...,5-11 pinnate
Amyris ignea,tubular,yellow,None of the above,None of the above,crenate,panicle,None of the above,translucent oil cells,None of the above (Information about the Stipu...,4-5,...,None of the above,elliptic,globose,None of the above,5,10,None of the above (the 'Leaf lower side' trait...,capsule,None of the above. The text did not mention an...,5-11 pinnate
Amyris ignea,tubular,yellow,None of the above,None of the above,crenate,panicle,None of the above,translucent oil cells,None of the above (Information about the Stipu...,4-5,...,None of the above,other: size and texture varied among species,ovoid,None of the above,5,10,None of the above (the 'Leaf lower side' trait...,capsule,None of the above. The text did not mention an...,5-11 pinnate
Amyris ignea,tubular,yellow,None of the above,None of the above,crenate,panicle,None of the above,translucent oil cells,None of the above (Information about the Stipu...,4-5,...,None of the above,other: size and texture varied among species,globose,None of the above,5,10,None of the above (the 'Leaf lower side' trait...,capsule,None of the above. The text did not mention an...,5-11 pinnate
Amyris ignea,tubular,yellow,None of the above,None of the above,crenate,panicle,None of the above,translucent oil cells,None of the above (Information about the Stipu...,4-5,...,None of the above,elliptic,ovoid,None of the above,5,10,None of the above (the 'Leaf lower side' trait...,capsule,None of the above. The text did not mention an...,5-11 pinnate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Bourreria succulenta,salverform,white,connate,alternate,entire,cymes,None of the above,absent,present,5 fused sepals forming a tube'] -> ['5,...,not winged,spatulate,berry that turns orange-red at maturity,alternate with the petals,5,5,smooth,berry,red,entire
Bourreria succulenta,salverform,white,connate,alternate,entire,cymes,None of the above,absent,present,5 fused sepals forming a tube'] -> ['5,...,not winged,spatulate,berry that turns orange-red at maturity,alternate with the petals,5,5,pale green,berry,orange,entire
Bourreria succulenta,salverform,white,connate,alternate,entire,cymes,None of the above,absent,present,5 fused sepals forming a tube'] -> ['5,...,not winged,spatulate,berry that turns orange-red at maturity,alternate with the petals,5,5,pale green,berry,red,entire
Bourreria succulenta,salverform,white,connate,alternate,entire,cymes,None of the above,absent,present,5 fused sepals forming a tube'] -> ['5,...,not winged,spatulate,berry that turns orange-red at maturity,alternate with the petals,5,5,pubescent,berry,orange,entire


In [102]:
df.columns

Index(['Petals corolla shape', 'Petals corolla colour', 'Sepals calyx shape',
       'Leaf position', 'Leaf margin', 'Inflorescence type', 'Seed colour',
       'Leaf glands', 'Stipules', 'Sepals calyx numer', 'Fruit colour',
       'Leaf upper side', 'Thornsspines', 'Life form', 'Leaf rachis',
       'Leaf shape', 'Fruit shape', 'Stamen shape', 'Petals corolla number',
       'Stamen number', 'Leaf lower side', 'Fruit type', 'Aril colour',
       'Leaf composition'],
      dtype='object')

In [105]:
dummies = pd.get_dummies(df, prefix=df.columns, prefix_sep='')
dummies

,Petals corolla shapeNone of the above,Petals corolla shaperounded,Petals corolla shapesalverform,Petals corolla shapetubular,Petals corolla colourNone of the above,Petals corolla colourpink,Petals corolla colourpurple,Petals corolla colourwhite,Petals corolla colouryellow,Sepals calyx shapeNone of the above,...,Leaf compositionOpposite,Leaf compositiondark green above,Leaf compositionelliptical to obovate,Leaf compositionentire,Leaf compositionevergreen,Leaf compositionleathery,Leaf compositionmay have salt deposits on their upper surfaces due to salt excretion.,Leaf compositionodd-pinnately compound,Leaf compositionsilvery hairy below,Leaf compositionsimple
Amyris ignea,0,0,0,1,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
Amyris ignea,0,0,0,1,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
Amyris ignea,0,0,0,1,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
Amyris ignea,0,0,0,1,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
Amyris ignea,0,0,0,1,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Bourreria succulenta,0,0,1,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
Bourreria succulenta,0,0,1,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
Bourreria succulenta,0,0,1,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
Bourreria succulenta,0,0,1,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
